# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 19 2022 9:06AM,244539,15,VT80176746,"Virtus Pharmaceuticals, LLC",Released
1,Aug 19 2022 9:03AM,244538,15,30010080,"Alliance Pharma, Inc.",Released
2,Aug 19 2022 9:03AM,244538,15,30010081,"Alliance Pharma, Inc.",Released
3,Aug 19 2022 9:03AM,244538,15,30010084,"Alliance Pharma, Inc.",Released
4,Aug 19 2022 9:03AM,244538,15,30010085,"Alliance Pharma, Inc.",Released
5,Aug 19 2022 9:03AM,244538,15,30010082,"Alliance Pharma, Inc.",Released
6,Aug 19 2022 9:03AM,244538,15,30010083,"Alliance Pharma, Inc.",Released
7,Aug 19 2022 9:03AM,244538,15,30010086,"Alliance Pharma, Inc.",Released
8,Aug 19 2022 9:03AM,244538,15,30010087,"Alliance Pharma, Inc.",Released
9,Aug 19 2022 9:03AM,244538,15,30010089,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,244535,Released,1
22,244536,Released,1
23,244537,Executing,1
24,244538,Released,16
25,244539,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
244535,NaN,NaN,1.0
244536,NaN,NaN,1.0
244537,NaN,1.0,NaN
244538,NaN,NaN,16.0
244539,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244446,1.0,17.0,11.0
244450,0.0,0.0,1.0
244454,0.0,0.0,1.0
244455,0.0,1.0,0.0
244457,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244446,1,17,11
244450,0,0,1
244454,0,0,1
244455,0,1,0
244457,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244446,1,17,11
1,244450,0,0,1
2,244454,0,0,1
3,244455,0,1,0
4,244457,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244446,1,17,11
1,244450,,,1
2,244454,,,1
3,244455,,1,
4,244457,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 19 2022 9:06AM,244539,15,"Virtus Pharmaceuticals, LLC"
1,Aug 19 2022 9:03AM,244538,15,"Alliance Pharma, Inc."
17,Aug 19 2022 8:35AM,244537,22,"NBTY Global, Inc."
18,Aug 19 2022 8:26AM,244536,10,Beach Patient Assistance
19,Aug 19 2022 8:22AM,244535,10,"Amcyte Pharma, Inc."
20,Aug 19 2022 8:19AM,244534,10,Emerginnova
22,Aug 19 2022 8:15AM,244531,10,ISDIN Corporation
37,Aug 19 2022 8:14AM,244532,19,"AdvaGen Pharma, Ltd"
39,Aug 19 2022 8:12AM,244530,10,ISDIN Corporation
46,Aug 19 2022 8:06AM,244528,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 19 2022 9:06AM,244539,15,"Virtus Pharmaceuticals, LLC",,,1
1,Aug 19 2022 9:03AM,244538,15,"Alliance Pharma, Inc.",,,16
2,Aug 19 2022 8:35AM,244537,22,"NBTY Global, Inc.",,1,
3,Aug 19 2022 8:26AM,244536,10,Beach Patient Assistance,,,1
4,Aug 19 2022 8:22AM,244535,10,"Amcyte Pharma, Inc.",,,1
5,Aug 19 2022 8:19AM,244534,10,Emerginnova,,,2
6,Aug 19 2022 8:15AM,244531,10,ISDIN Corporation,,,15
7,Aug 19 2022 8:14AM,244532,19,"AdvaGen Pharma, Ltd",,2,
8,Aug 19 2022 8:12AM,244530,10,ISDIN Corporation,,,7
9,Aug 19 2022 8:06AM,244528,10,ISDIN Corporation,,,13


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 19 2022 9:06AM,244539,15,"Virtus Pharmaceuticals, LLC",1,,
1,Aug 19 2022 9:03AM,244538,15,"Alliance Pharma, Inc.",16,,
2,Aug 19 2022 8:35AM,244537,22,"NBTY Global, Inc.",,1,
3,Aug 19 2022 8:26AM,244536,10,Beach Patient Assistance,1,,
4,Aug 19 2022 8:22AM,244535,10,"Amcyte Pharma, Inc.",1,,
5,Aug 19 2022 8:19AM,244534,10,Emerginnova,2,,
6,Aug 19 2022 8:15AM,244531,10,ISDIN Corporation,15,,
7,Aug 19 2022 8:14AM,244532,19,"AdvaGen Pharma, Ltd",,2,
8,Aug 19 2022 8:12AM,244530,10,ISDIN Corporation,7,,
9,Aug 19 2022 8:06AM,244528,10,ISDIN Corporation,13,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 19 2022 9:06AM,244539,15,"Virtus Pharmaceuticals, LLC",1,,
1,Aug 19 2022 9:03AM,244538,15,"Alliance Pharma, Inc.",16,,
2,Aug 19 2022 8:35AM,244537,22,"NBTY Global, Inc.",,1,
3,Aug 19 2022 8:26AM,244536,10,Beach Patient Assistance,1,,
4,Aug 19 2022 8:22AM,244535,10,"Amcyte Pharma, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 19 2022 9:06AM,244539,15,"Virtus Pharmaceuticals, LLC",1.0,NaN,NaN
1,Aug 19 2022 9:03AM,244538,15,"Alliance Pharma, Inc.",16.0,NaN,NaN
2,Aug 19 2022 8:35AM,244537,22,"NBTY Global, Inc.",NaN,1.0,NaN
3,Aug 19 2022 8:26AM,244536,10,Beach Patient Assistance,1.0,NaN,NaN
4,Aug 19 2022 8:22AM,244535,10,"Amcyte Pharma, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 19 2022 9:06AM,244539,15,"Virtus Pharmaceuticals, LLC",1.0,0.0,0.0
1,Aug 19 2022 9:03AM,244538,15,"Alliance Pharma, Inc.",16.0,0.0,0.0
2,Aug 19 2022 8:35AM,244537,22,"NBTY Global, Inc.",0.0,1.0,0.0
3,Aug 19 2022 8:26AM,244536,10,Beach Patient Assistance,1.0,0.0,0.0
4,Aug 19 2022 8:22AM,244535,10,"Amcyte Pharma, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2445009,53.0,17.0,1.0
102,244450,1.0,0.0,0.0
15,489077,17.0,0.0,0.0
16,978047,5.0,1.0,0.0
17,244455,0.0,1.0,0.0
19,978079,2.0,3.0,0.0
20,244459,17.0,0.0,0.0
22,244537,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2445009,53.0,17.0,1.0
1,102,244450,1.0,0.0,0.0
2,15,489077,17.0,0.0,0.0
3,16,978047,5.0,1.0,0.0
4,17,244455,0.0,1.0,0.0
5,19,978079,2.0,3.0,0.0
6,20,244459,17.0,0.0,0.0
7,22,244537,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,53.0,17.0,1.0
1,102,1.0,0.0,0.0
2,15,17.0,0.0,0.0
3,16,5.0,1.0,0.0
4,17,0.0,1.0,0.0
5,19,2.0,3.0,0.0
6,20,17.0,0.0,0.0
7,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,53.0
1,102,Released,1.0
2,15,Released,17.0
3,16,Released,5.0
4,17,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,15,16,17,19,20,22
Status,,,,,,,,
Completed,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,17.0,0.0,0.0,1.0,1.0,3.0,0.0,1.0
Released,53.0,1.0,17.0,5.0,0.0,2.0,17.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,15,16,17,19,20,22
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,17.0,0.0,0.0,1.0,1.0,3.0,0.0,1.0
2,Released,53.0,1.0,17.0,5.0,0.0,2.0,17.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,15,16,17,19,20,22
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,17.0,0.0,0.0,1.0,1.0,3.0,0.0,1.0
2,Released,53.0,1.0,17.0,5.0,0.0,2.0,17.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()